## **KT analysis**
*Dan Farbowitz*

A note book to explore clearness index calculation and distributions using pvlib.
Originally designed by Jamie Taylor, A Buckley

First Authored: 2019-11-05

Added on importing large data sets, plots, and regressions.

Revised: 2020-02-12


Install python package

In [ ]:
!pip install numpy pandas>1.0 matplotlib pvlib geopy geocoder

Import modules. force_remount is used because of the need to import from different folders as a result of the number of files involved.

In [ ]:
from datetime import datetime as dt

from google.colab import drive
drive.mount('drive', force_remount=True)

import pytz
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pvlib
from geopy.geocoders import Nominatim
import glob
import os
import json
import requests
import scipy.stats as sp
from sklearn.linear_model import LinearRegression
import statsmodels.formula.api as sm 
from statsmodels.stats.outliers_influence import summary_table
from statsmodels.sandbox.regression.predstd import wls_prediction_std
pd.set_option('display.max_rows', None)


A previous variant used geopy to reverse-lookup the location from its (lat, long) coordinates given in the NREL data.

Temperature will be averaged from temperature data later.

Due to long wait times in compiling data, split it up by data set.

In [ ]:
#default timezone
tz = "UTC"
#New Import Method for NSRDB data
folders = ['/content/drive/MyDrive/KT data/NREL NSRDB Data/Dataset A - Near Boston','/content/drive/MyDrive/KT data/NREL NSRDB Data/Dataset B - Bay Area','/content/drive/MyDrive/KT data/NREL NSRDB Data/Dataset C - Death Valley','/content/drive/MyDrive/KT data/NREL NSRDB Data/Dataset D - NE of Albaquerque','/content/drive/MyDrive/KT data/NREL NSRDB Data/Dataset E - NW of Des Moines','/content/drive/MyDrive/KT data/NREL NSRDB Data/Dataset F - Big Island of Hawaii' ]



cells = {}
elev_data = {}
for folder in folders:
  all_files = glob.glob(os.path.join(folder, '*.csv'))
  for filename in all_files:
  
    #extracting relevant information from first 2 lines of original file (necessary because of oddness in NREL data files that gave overflow errors)
    paraminfo = pd.read_csv(filename, sep = ',', usecols = ['Elevation', 'Latitude', 'Longitude'], nrows=1)
    df = pd.read_csv(filename, header=2)
    #add in positional information to dataframe
    elevation = paraminfo['Elevation'][0]
    latitude = paraminfo['Latitude'][0]
    longitude = paraminfo['Longitude'][0]
    #elimniating extra 'Unnamed' columns
    for i in df.columns:
      if i[:2] == 'Un':
        df = df.drop([i], axis=1)
    #replacing date/time columns with datetime index
    df.index = pd.to_datetime(df[df.columns[0:5]])
    df = df.drop(['Year', 'Month', 'Day', 'Hour', 'Minute'], axis=1)
    #data merging
    if (str(latitude) + "," + str(longitude)) not in cells:
      cells[str(latitude) + "," + str(longitude)] = df
      elev_data[str(latitude) + "," + str(longitude)] = int(elevation)
    else:
      cells[str(latitude) + "," + str(longitude)] = pd.concat([cells[str(latitude) + "," + str(longitude)], df])

#and sorting
for cell in cells:
  cells[cell] = cells[cell].sort_index()
#double-check that elevation data is correct
print(elev_data)
  

Creating a function to return the name of the time interval for labeling.

In [ ]:
#time-based indices
def t_alt(short):
  if short == 'H':
    return 'Hourly'
  elif short == 'D':
    return 'Daily'
  elif short == 'W':
    return 'Weekly'
  elif short == 'M':
    return 'Monthly'
  elif short == 'Q':
    return 'Quarterly'
  elif short == 'Y':
    return 'Annual'
  else:
    return ''

etr is the extraterrestrial solar radiation for a given time - but it is not corrected to the horizontal plane (etr_hor) at a particular latitude and longitude. To do this correction we need to use the cosine of the zenith angle (from solpos) at that position.

In [ ]:
#looping data over time periods
def get_KT_Charts(dataset, lat, long, elev, interval): 
  #setting up info
  latitude = lat
  longitude = long
  suptl = str(lat) + "," + str(long)
  dataset = dataset.tz_localize('UTC')
  temperature = dataset['Temperature'].mean()
  start = dataset.index[0]
  end = dataset.index[-1]
  altitude = elev

  #using available information to get solar position
  times = pd.date_range(start = start, end = end, tz = tz, freq = 'H')
  loc = pvlib.location.Location(latitude = latitude, longitude = longitude, altitude=altitude)
  solpos = loc.get_solarposition(times, temperature)
  #calculating irradiance from solar position, including horizontal component by taking cos(zenith angle) of irradiance data
  etr = pvlib.irradiance.get_extra_radiation(times)
  zen = solpos["apparent_zenith"]
  etr_hor = np.cos(np.radians(zen)) * etr
  #setting negative values (nighttime 'irradiance') to 0
  etr_hor[etr_hor < 0] = 0
  #taking means over time interval
  etr_h_hor = etr_hor.resample(interval, label="right").mean()
  irr = dataset.GHI.resample(interval, label = 'right').mean()
  #define KT, removing out of range samples (and 0 so nighttime doesn't overwhelm histogram)
  kt = irr / etr_h_hor
  kt[kt > 1.3] = np.nan
  kt[kt == 0] = np.nan
  #relabel for graphs
  t_int = t_alt(interval)
  #plotting 3 graphs in a 2x2 grid
  grid = plt.GridSpec(2, 2, wspace=0.4, hspace=0.4)
  plt.subplot(grid[0,:2])
  irr.plot(ylabel='Horizontal Irradiance (W/m^2)', xlabel='Date/Time', figsize = (10, 10))
  etr_h_hor.plot(ylabel='Horizontal Irradiance (W/m^2)', xlabel='Date/Time')
  plt.legend(labels=['Global', 'Extraterrestrial'])

  #KT data
  plt.subplot(grid[1, 0])
  kt.plot(ylabel='KT Index', xlabel='Date/Time', ylim = (0, 1.4))
  
  #KT Histogram
  plt.subplot(grid[1,1])
  plt.hist(kt[kt.notnull()], bins=130, range=(0,1.3))
  plt.xlabel('KT Index')
  plt.ylabel('Number of occurences')
  
  plt.suptitle(suptl + ' ' + t_int)
  plt.show()

#call up charts for each time interval based on location (not used)
def KT_loc(data):
  t_index = ['H', 'D', 'W', 'M', 'Q']
  for j in range(5):
    get_KT_Charts(data, t_index[j])

#run for all data to check if it matches reasonably
for i in cells:
  get_KT_Charts(cells[i], float(i.split(",")[0]), float(i.split(",")[1]), float(elev_data[i]), 'W')

Using the same method to put KT data together for regression:


In [ ]:
def KT_data(dataset, lat, long, altitude, interval): 
  #setting up info
  latitude = lat
  longitude = long
  dataset = dataset.tz_localize('UTC')
  temperature = dataset['Temperature'].mean()
  start = dataset.index[0]
  end = dataset.index[-1]
  #using available information to get solar position
  times = pd.date_range(start = start, end = end, tz = tz, freq = "H")
  loc = pvlib.location.Location(latitude = latitude, longitude = longitude, altitude=altitude)
  solpos = loc.get_solarposition(times=times, temperature=temperature)
  #calculating irradiance from solar position, including horizontal component by taking cos(zenith angle) of irradiance data
  etr = pvlib.irradiance.get_extra_radiation(times)
  zen = solpos["apparent_zenith"]
  etr_hor = np.cos(np.radians(zen)) * etr
  #setting negative values (nighttime 'irradiance') to 0
  etr_hor[etr_hor < 0] = 0
  #taking means over time interval
  etr_h_hor = etr_hor.resample(interval, label="right").mean()
  irr = dataset.GHI.resample(interval, label = 'right').mean()
  #define KT, removing out of range samples (and 0 so nighttime doesn't overwhelm histogram)
  kt = irr / etr_h_hor
  kt[kt > 1.3] = np.nan
  kt[kt == 0] = np.nan
  df = pd.DataFrame({'kt': kt})
  return df





In [ ]:
#creating KT data for each cell
kt_cells = {}
for i in cells:
   kt_cells[i] = KT_data(cells[i], float(i.split(",")[0]), float(i.split(",")[1]), float(elev_data[i]), 'D')
   #DAS = Days after start, changing to numeric values for regression
   kt_cells[i]['DAS'] = (kt_cells[i].index - pd.Timestamp("1998-01-01", tz='UTC')).days



In [ ]:
#KT analysis


def kt_analysis(dataset, lat, long, interval): 
  #setting up info
  latitude = lat
  longitude = long
  dataset = dataset.tz_convert('UTC')
  kt = dataset['kt']
  start = dataset.index[0]
  end = dataset.index[-1]
  #taking means over time interval
  kt_avg = kt.resample(interval, label="right").mean()
  kt_avg = pd.DataFrame(kt_avg, columns=['kt'])
  #Again using days after start for regression
  kt_avg['DAS'] = (kt_avg.index - pd.Timestamp("1998-01-01", tz='UTC')).days

  # Following method derived from https://stackoverflow.com/questions/34998772/plotting-confidence-and-prediction-intervals-with-repeated-entries, adapted for linear regression model
  #plot with regression line  
  x = kt_avg.DAS
  y = kt_avg.kt

  plt.figure(figsize=(6 * 1.618, 6))
  plt.scatter(x,y, s=10, alpha=0.3)
  plt.xlabel('Days since 01-01-1998')
  plt.ylabel('KT')

  # points linearly spaced for predictor variable
  x1 = pd.DataFrame({'DAS': np.linspace(kt_avg.DAS.min(), kt_avg.DAS.max(), 100)})


  linreg = sm.ols(formula='kt ~ DAS',   data=kt_avg).fit()

  # Plots linear bestfit line:


  prstd, iv_l, iv_u = wls_prediction_std(linreg)

  st, data, ss2 = summary_table(linreg, alpha=0.05)

  fittedvalues = data[:,2]
  predict_mean_se  = data[:,3]
  predict_mean_ci_low, predict_mean_ci_upp = data[:,4:6].T
  predict_ci_low, predict_ci_upp = data[:,6:8].T
  #data
  plt.plot(x, y, 'o')
  #linear regression
  plt.plot(x1.DAS, linreg.predict(x1), '-', label='Linear Fit  $R^2$=%.2f' % linreg.rsquared)
  #prediction interval based on slope
  plt.plot(x, predict_mean_ci_low, 'r--', lw=2, label='95% confidence interval')
  plt.plot(x, predict_mean_ci_upp, 'r--', lw=2)
  #confidence interval based on both coefficients
  plt.plot(x, predict_ci_low, 'g--', lw=2, label='95% prediction interval')
  plt.plot(x, predict_ci_upp, 'g--', lw=2)
  plt.title("("+str(lat)+","+str(long)+") "+t_alt(interval)+ " sampling")


  plt.legend()


  #Commented out, but helps verify statistics
  #print(sm.ols(formula = "kt ~ DAS", data=kt_avg).fit().summary())
  out = {'Location': "("+str(lat)+", "+str(long)+")", 'Estimated per year change to KT': sm.ols(formula = "kt ~ DAS", data=kt_avg).fit().params['DAS']*365.25, 'p-value': linreg.pvalues['DAS'], 'r^2-value':linreg.rsquared}
  return out


#runs kt_analysis to form a chart of useful value

kt_data = pd.DataFrame(columns=['Location', 'Estimated per year change to KT', 'p-value', 'r^2-value'])
for i in kt_cells:
  kt_data = kt_data.append(kt_analysis(kt_cells[i], float(i.split(",")[0]), float(i.split(",")[1]), 'Y'), ignore_index=True)
kt_data = kt_data.set_index('Location')
kt_data

In [ ]:
#temperature analysis
temp_cells = {}

def temp_analysis(dataset, lat, long, interval): 
  #setting up info
  latitude = lat
  longitude = long
  dataset = dataset.tz_localize('UTC')
  temperature = dataset['Temperature']
  start = dataset.index[0]
  end = dataset.index[-1]
  #taking means over time interval
  temps = temperature.resample(interval, label="right").mean()
  temps = pd.DataFrame(temps, columns=['Temperature'])
  #Again using days after start for regression
  temps['DAS'] = (temps.index - pd.Timestamp("1998-01-01", tz='UTC')).days

  # Following method derived from https://stackoverflow.com/questions/34998772/plotting-confidence-and-prediction-intervals-with-repeated-entries, adapted for linear regression model
  #plot with regression line  
  x = temps.DAS
  y = temps.Temperature

  plt.figure(figsize=(6 * 1.618, 6))
  plt.scatter(x,y, s=10, alpha=0.3)
  plt.xlabel('Days since 01-01-1998')
  plt.ylabel('Temperature (degC)')

  # points linearly spaced for predictor variable
  x1 = pd.DataFrame({'DAS': np.linspace(temps.DAS.min(), temps.DAS.max(), 100)})


  linreg = sm.ols(formula='Temperature ~ DAS',   data=temps).fit()

  # Plots linear bestfit line:


  prstd, iv_l, iv_u = wls_prediction_std(linreg)

  st, data, ss2 = summary_table(linreg, alpha=0.05)

  fittedvalues = data[:,2]
  predict_mean_se  = data[:,3]
  predict_mean_ci_low, predict_mean_ci_upp = data[:,4:6].T
  predict_ci_low, predict_ci_upp = data[:,6:8].T
  #data
  plt.plot(x, y, 'o')
  #linear regression
  plt.plot(x1.DAS, linreg.predict(x1), '-', label='Linear Fit  $R^2$=%.2f' % linreg.rsquared)
  #prediction interval based on slope
  plt.plot(x, predict_mean_ci_low, 'r--', lw=2, label='95% confidence interval')
  plt.plot(x, predict_mean_ci_upp, 'r--', lw=2)
  #confidence interval based on both coefficients
  plt.plot(x, predict_ci_low, 'g--', lw=2, label='95% prediction interval')
  plt.plot(x, predict_ci_upp, 'g--', lw=2)
  plt.title("("+str(lat)+","+str(long)+") "+t_alt(interval)+ " sampling")


  plt.legend()


  #Commented out, but helps verify statistics
  #print(sm.ols(formula = "Temperature ~ DAS", data=temps).fit().summary())
  out = {'Location': "("+str(lat)+", "+str(long)+")", 'Estimated per year change (degC)': sm.ols(formula = "Temperature ~ DAS", data=temps).fit().params['DAS']*365.25, 'p-value': linreg.pvalues['DAS'], 'r^2-value':linreg.rsquared}
  return out


#runs temp_analysis to form a chart of useful value
temp_data = pd.DataFrame(columns=['Location', 'Estimated per year change (degC)', 'p-value', 'r^2-value'])
for i in cells:
   temp_data = temp_data.append(temp_analysis(cells[i], float(i.split(",")[0]), float(i.split(",")[1]), 'M'), ignore_index=True)
temp_data = temp_data.set_index('Location')
temp_data